In [ ]:
'''Q1 Classification via Generative Models'''

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import pickle 

with open('phone_train.pickle', 'rb') as fh1:
    traindata = pickle.load(fh1)
    
print("The first few rows are:")
print(traindata.head())
print("\nSummary statistics:")
print(traindata.describe())
print("\nLabel counts:")
print(traindata['label'].value_counts())

with open('phone_test1.pickle', 'rb') as fh2:
    testdata = pickle.load(fh2)


The first few rows are:
          x         y         z        label
0  0.138809  0.074341  9.801056  Phoneonback
1  0.164993  0.006500  9.690369  Phoneonback
2  0.211411 -0.001831  9.755829  Phoneonback
3  0.184036 -0.007782  9.774872  Phoneonback
4  0.142380  0.005310  9.765350  Phoneonback

Summary statistics:
                   x              y              z
count  167097.000000  167097.000000  167097.000000
mean        0.357340       0.146807      -0.015550
std         5.622396       5.552010       5.737150
min        -9.770279      -9.966507      -9.908417
25%         0.016617      -0.074875      -0.221497
50%         0.142776       0.009628       0.025223
75%         0.249893       0.295715       0.151032
max        10.073685       9.980255      10.031113

Label counts:
Phoneonleft      29522
Phoneonfront     29079
Phoneonback      28566
Phoneonbottom    27842
Phoneontop       26401
Phoneonright     25687
Name: label, dtype: int64


In [2]:
X_train = pd.concat([traindata['x'], traindata['y'], traindata['z']], axis = 1)
Y_train = traindata['label']
X_test = pd.concat([testdata['x'], testdata['y'], testdata['z']], axis = 1)
Y_test = testdata['label']

In [3]:
# create mypgc class
import math

class mypgc:
    
    def __init__(self):
        self.X_train = []
        self.Y_train = []
        self.locations = ['Phoneonleft','Phoneonfront', 'Phoneonback', 'Phoneonbottom', 'Phoneontop', 'Phoneonright']
        
    def fit(self, x_train, y_train):
        
        self.X_train = x_train
        self.Y_train = y_train
        self.traindata = pd.concat([x_train, y_train], axis = 1)
        
        self.amodel = {}
        for loc in self.locations:
            self.amodel[loc] = {}
            mu = np.array(self.traindata.loc[self.traindata['label'] == loc].mean())
            cov = np.array(self.traindata.loc[self.traindata['label'] == loc].cov())
            prec = np.linalg.pinv(cov)
            sign, detcov = np.linalg.slogdet(cov)
            n = (self.traindata.loc[self.traindata['label'] == loc]).count()
            prior = (self.traindata.loc[self.traindata['label'] == loc]).count() / self.traindata.count()
            self.amodel[loc]['mu'] = mu
            self.amodel[loc]['cov'] = cov
            self.amodel[loc]['prec'] = prec
            self.amodel[loc]['detcov'] = sign*detcov
            self.amodel[loc]['n'] = n
            self.amodel[loc]['prior'] = prior
        self.show_amodel()
        
    def predict(self, x_test):
        self.X_test = x_test.values
        pi = math.pi
        ypred = []
        for i in range(len(self.X_test)): 
            prob_kg_opt = 0
            y = ''
            for loc in self.locations:
                #prob_gk = 0.05
                exp = np.exp((-1/2)*np.transpose(self.X_test[i]-self.amodel[loc]['mu']).dot(self.amodel[loc]['prec']).dot(self.X_test[i]-self.amodel[loc]['mu']))
                prob_gk = (1/((2*pi)**(3/2)))*(1/np.sqrt(abs(self.amodel[loc]['detcov'])))*exp
                prob_kg = self.amodel[loc]['prior'][0]*prob_gk 
                
                if prob_kg > prob_kg_opt:
                    prob_kg_opt = prob_kg
                    y = loc
            ypred.append(y)
            
        return ypred
    # List my learned model parameters  
    def show_amodel(self):
        for l in self.locations:
            print(l)
            for i in self.amodel[l]:
                if type(self.amodel[l][i])==type(np.array([])):
                    print("%s: \n"%i,str(self.amodel[l][i]))
                else:
                    print("%s: "%i,str(self.amodel[l][i]))
            print("-------------------------------------------------------------")
    

In [4]:
#  apply my predict method. 
pgc1 = mypgc()
pgc1.fit(X_train, Y_train)
ypred = pgc1.predict(X_test)

Phoneonleft
mu: 
 [ 9.92639446  0.09275717 -0.01933473]
cov: 
 [[ 0.00192839 -0.00619274  0.00063551]
 [-0.00619274  0.03200844 -0.00317611]
 [ 0.00063551 -0.00317611  0.00171996]]
prec: 
 [[1369.95113577  263.0134811   -20.49603676]
 [ 263.0134811    88.74589895   66.6992188 ]
 [ -20.49603676   66.6992188   712.14869845]]
detcov:  -17.232141935734493
n:  x        29522
y        29522
z        29522
label    29522
dtype: int64
prior:  x        0.176676
y        0.176676
z        0.176676
label    0.176676
dtype: float64
-------------------------------------------------------------
Phoneonfront
mu: 
 [ 0.11270401  0.14265129 -9.73579633]
cov: 
 [[ 0.00173617 -0.00562249  0.00071319]
 [-0.00562249  0.03069977 -0.00386164]
 [ 0.00071319 -0.00386164  0.00185582]]
prec: 
 [[1415.5956848   258.4845593    -6.15206605]
 [ 258.4845593    91.32078358   90.68682325]
 [  -6.15206605   90.68682325  729.91124637]]
detcov:  -17.331692277173122
n:  x        29079
y        29079
z        29079
label   

In [5]:
# list the first 20 predictions and well as the correct labels. Compute the accuracy for your model.
print('ypred:\n', ypred[0:20])
print('yreal:\n', Y_test[0:20])
print('accuracy = ', np.sum(ypred == Y_test)/len(Y_test))

ypred:
 ['Phoneonfront', 'Phoneonbottom', 'Phoneonbottom', 'Phoneonbottom', 'Phoneonfront', 'Phoneonright', 'Phoneonfront', 'Phoneontop', 'Phoneonfront', 'Phoneonfront', 'Phoneonfront', 'Phoneonback', 'Phoneonleft', 'Phoneonback', 'Phoneonbottom', 'Phoneonback', 'Phoneonright', 'Phoneonright', 'Phoneontop', 'Phoneonleft']
yreal:
 0      Phoneonfront
1     Phoneonbottom
2     Phoneonbottom
3     Phoneonbottom
4      Phoneonfront
5      Phoneonright
6      Phoneonfront
7        Phoneontop
8      Phoneonfront
9      Phoneonfront
10     Phoneonfront
11      Phoneonback
12      Phoneonleft
13      Phoneonback
14    Phoneonbottom
15      Phoneonback
16     Phoneonright
17     Phoneonright
18       Phoneontop
19      Phoneonleft
Name: label, dtype: object
accuracy =  1.0


In [ ]:
'''Q2 Logistic Regression with L2 Regularization'''

In [ ]:
'''
Q2.1 (20%) Download the Adult dataset. 
Clean up the dataset and create x_train, y_train, x_test, y_test for training feature, training value, test feature, test label.
All of these variables should be numpy arrays. 
Provide summary statistics for your training and test datasets so that TA can verify the correctness of your procedure.
'''

In [6]:
import numpy as np
import pandas as pd

namelist = ['age', 'workclass', 'fnlwgt', 'edu', 'edu-num', 'marital', 'occ', 'rel', 'race', 'sex', 'cgain','closs','hpw', 'ncountry','income']
traindata = pd.read_csv('adult.data', names = namelist)
traindata = traindata.dropna(axis = 0)
for name in namelist:
    traindata.drop(traindata.loc[traindata[name]==' ?'].index, inplace=True)

testdata = pd.read_csv('adult.test', names = namelist)
testdata = testdata.dropna(axis = 0)
for name in namelist:
    testdata.drop(testdata.loc[testdata[name]==' ?'].index, inplace=True)


C:\Users\cherc\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [7]:
traindata

,age,workclass,fnlwgt,edu,edu-num,marital,occ,rel,race,sex,cgain,closs,hpw,ncountry,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [8]:
 '''
 "1-of-K" encoding, 
 Include a particular feature value only if this unique value appears more than 10 times in the training data.
 '''
# 檢查看看哪幾個feature value出現低於10次
categories = ['workclass','edu','marital','occ','rel','race','sex','ncountry']
print(traindata['workclass'].value_counts())
print(traindata['edu'].value_counts())
print(traindata['marital'].value_counts())
print(traindata['occ'].value_counts())
print(traindata['rel'].value_counts())
print(traindata['race'].value_counts())
print(traindata['sex'].value_counts())
print(traindata['ncountry'].value_counts())


 Private             22286
 Self-emp-not-inc     2499
 Local-gov            2067
 State-gov            1279
 Self-emp-inc         1074
 Federal-gov           943
 Without-pay            14
Name: workclass, dtype: int64
 HS-grad         9840
 Some-college    6678
 Bachelors       5044
 Masters         1627
 Assoc-voc       1307
 11th            1048
 Assoc-acdm      1008
 10th             820
 7th-8th          557
 Prof-school      542
 9th              455
 12th             377
 Doctorate        375
 5th-6th          288
 1st-4th          151
 Preschool         45
Name: edu, dtype: int64
 Married-civ-spouse       14065
 Never-married             9726
 Divorced                  4214
 Separated                  939
 Widowed                    827
 Married-spouse-absent      370
 Married-AF-spouse           21
Name: marital, dtype: int64
 Prof-specialty       4038
 Craft-repair         4030
 Exec-managerial      3992
 Adm-clerical         3721
 Sales                3584
 Other-service    

In [9]:
# encoding
# training data
wc_enc = pd.get_dummies(traindata['workclass'], prefix = 'workclass')
edu_enc = pd.get_dummies(traindata['edu'], prefix = 'edu')
mt_enc = pd.get_dummies(traindata['marital'], prefix = 'marital')
occ_enc = pd.get_dummies(traindata['occ'], prefix = 'occ')
rel_enc = pd.get_dummies(traindata['rel'], prefix = 'rel')
race_enc = pd.get_dummies(traindata['race'], prefix = 'race')
sex_enc = pd.get_dummies(traindata['sex'], prefix = 'sex')
nc_enc = pd.get_dummies(traindata['ncountry'], prefix = 'ncountry')

traindata_enc = pd.concat([traindata.drop(columns=categories), wc_enc, edu_enc, mt_enc, occ_enc, rel_enc, race_enc, sex_enc, nc_enc], axis = 1)
traindata_enc = traindata_enc.drop('occ_ Armed-Forces', 1)
traindata_enc = traindata_enc.drop('ncountry_ Holand-Netherlands', 1)

# test data
wc_enc = pd.get_dummies(testdata['workclass'], prefix = 'workclass')
edu_enc = pd.get_dummies(testdata['edu'], prefix = 'edu')
mt_enc = pd.get_dummies(testdata['marital'], prefix = 'marital')
occ_enc = pd.get_dummies(testdata['occ'], prefix = 'occ')
rel_enc = pd.get_dummies(testdata['rel'], prefix = 'rel')
race_enc = pd.get_dummies(testdata['race'], prefix = 'race')
sex_enc = pd.get_dummies(testdata['sex'], prefix = 'sex')
nc_enc = pd.get_dummies(testdata['ncountry'], prefix = 'ncountry')

testdata_enc = pd.concat([testdata.drop(columns=categories), wc_enc, edu_enc, mt_enc, occ_enc, rel_enc, race_enc, sex_enc, nc_enc], axis = 1)
testdata_enc = testdata_enc.drop('occ_ Armed-Forces', 1)

In [10]:
traindata_enc.describe()

,age,fnlwgt,edu-num,cgain,closs,hpw,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,...,ncountry_ Portugal,ncountry_ Puerto-Rico,ncountry_ Scotland,ncountry_ South,ncountry_ Taiwan,ncountry_ Thailand,ncountry_ Trinadad&Tobago,ncountry_ United-States,ncountry_ Vietnam,ncountry_ Yugoslavia
count,30162.000000,3.016200e+04,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,...,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000
mean,38.437902,1.897938e+05,10.121312,1092.007858,88.372489,40.931238,0.031265,0.068530,0.738877,0.035608,...,0.001127,0.003614,0.000365,0.002354,0.001392,0.000564,0.000597,0.911876,0.002122,0.000530
std,13.134665,1.056530e+05,2.549995,7406.346497,404.298370,11.979984,0.174035,0.252657,0.439254,0.185313,...,0.033556,0.060007,0.019094,0.048461,0.037291,0.023734,0.024422,0.283480,0.046016,0.023026
min,17.000000,1.376900e+04,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,1.176272e+05,9.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,37.000000,1.784250e+05,10.000000,0.000000,0.000000,40.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,47.000000,2.376285e+05,13.000000,0.000000,0.000000,45.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
testdata_enc['age'] = pd.to_numeric(testdata_enc['age']) 
testdata_enc.describe()

,age,fnlwgt,edu-num,cgain,closs,hpw,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,...,ncountry_ Portugal,ncountry_ Puerto-Rico,ncountry_ Scotland,ncountry_ South,ncountry_ Taiwan,ncountry_ Thailand,ncountry_ Trinadad&Tobago,ncountry_ United-States,ncountry_ Vietnam,ncountry_ Yugoslavia
count,15060.000000,1.506000e+04,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,...,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000
mean,38.768327,1.896164e+05,10.112749,1120.301594,89.041899,40.951594,0.030744,0.068592,0.731806,0.037981,...,0.001859,0.004382,0.000598,0.001992,0.000863,0.000797,0.000531,0.915538,0.001262,0.000465
std,13.380676,1.056150e+05,2.558727,7703.181842,406.283245,12.062831,0.172628,0.252768,0.443034,0.191158,...,0.043080,0.066057,0.024440,0.044589,0.029369,0.028218,0.023043,0.278089,0.035498,0.021555
min,17.000000,1.349200e+04,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,1.166550e+05,9.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,37.000000,1.779550e+05,10.000000,0.000000,0.000000,40.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,48.000000,2.385888e+05,13.000000,0.000000,0.000000,45.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,3770.000000,99.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
# create x_train, y_train, x_test, y_test as numpy arrays
X_train = traindata_enc.drop('income', 1).values
Y_train = traindata_enc['income'].values
X_test = testdata_enc.drop('income', 1).values
Y_test = testdata_enc['income'].values

for i in range(len(Y_train)):
    if Y_train[i] == ' >50K':
        Y_train[i] = 1
    else:
        Y_train[i] = 0
        
for i in range(len(Y_test)):
    if Y_test[i] == ' >50K':
        Y_test[i] = 1
    else:
        Y_test[i] = 0


In [ ]:
'''Q2.2 (20%) Derive the gradient and hession matrix for the new E(w)'''

In [15]:
dim = len(X_train[0])
lambda_vec = np.identity(dim)
b = np.sum(lambda_vec)/dim
w = np.dot(np.dot(np.linalg.pinv(np.dot(X_train.T,X_train)+b*np.identity(dim)),X_train.T),Y_train)
y = 1/(1+np.exp((-np.dot(X_train, w)).astype('int')))
rnn = np.identity(len(y))
np.fill_diagonal(rnn,y*(1-y))
hess = np.dot(np.dot(X_train.T, rnn),X_train)+lambda_vec
grad = np.dot(X_train.T,y-Y_train)+np.dot(lambda_vec,w)

print("gradient: \n", grad)
print("hession: \n", hess)

gradient: 
 [251621.56493283596 1457647860.2508235 66056.52790120582 -9918436.84339858
 -106268.03510196762 276290.95349012426 107.2472140649279
 425.60788716624955 6288.0110379566595 -51.407229775919255
 540.7320521989658 296.36269905117143 6.845847972425915 351.1763578031391
 464.95446083116474 159.45448304957736 69.49203185719246
 131.97671193892717 243.41390267175913 202.43136134412418
 247.92194295734043 309.6730097790745 405.0095473815216 -85.4395646837629
 3306.8499909469506 -99.35212343984165 22.531220661604216
 -121.48887389238519 2004.7950446917303 1656.526943253482
 0.6127723766670843 670.7200425029575 154.1725486048945 4394.056094583036
 403.8846835965739 333.4264186652305 1363.000591722179 1108.403712399357
 71.41060263035166 379.430882550909 591.9804962647298 738.2198190718942
 1474.4774408146175 70.55235966738637 227.74134501400837
 112.10950064002685 827.6234381444879 178.56922980847284
 467.22544033057767 586.7319291406866 3043.101907559788 409.39694781896424
 2168.892

In [ ]:
'''
Q2.3 (30%) Create your own mylogistic_l2 class. Show the learned w as well as test accuracy for the cases below. 
If w is too long for you, show selected w for continuous-valued, binary-valued, and the constant term. 
Case 1: lambda = 1 for all coefficients
Case 2: lambda = 1 for all but the intercept, no regularization for intercept term.
Case 3: lambda = 1 for numerical-valued features, lambda = 0.5 for binary-valued features, no regularization for incercept term.
'''

In [16]:
class mylogistic_l2:
    
    def __init__(self, reg_vec, max_iter, tol, add_intercept):
        self.X_train = []
        self.Y_train = []
        self.reg_vec = reg_vec
        self.max_iter = max_iter
        self.tol = tol
        self.add_intercept = add_intercept
        self.w = None
        
    def error(self, yreal, yprob):
        e=-1*np.sum(np.dot(yreal, np.log(yprob+1e-10))+np.dot((1-yreal.T), np.log(1-yprob+1e-10)))
        l2=0.5*(self.w.T).dot(self.reg_vec).dot(self.w)
        return e+l2
    
    def sigmoid(self,s):
        return 1/(1+np.exp(-s))
        
    def fit(self, x_train, y_train):
        self.X_train = x_train
        self.Y_train = y_train
        
        if self.add_intercept:
            self.X_train = np.concatenate((self.X_train, np.ones((self.X_train.shape[0],1))), axis=1)
        else:
            self.reg_vec = np.delete(self.reg_vec, np.s_[-1], axis=0)
            self.reg_vec = np.delete(self.reg_vec, np.s_[-1], axis=1)
 
        dim = len(self.X_train[0])
        b = np.sum(self.reg_vec)/dim
        self.w = np.linalg.inv((self.X_train.T).dot(self.X_train)+b*np.identity(dim)).dot(self.X_train.T).dot(self.Y_train)
        y = self.sigmoid((self.w.T).dot(self.X_train[0]))
        E_old = self.error(self.Y_train, y)
    
        for i in range(self.max_iter):
            ys = np.zeros(len(self.X_train))
            for v in range(len(self.X_train)):
                ys[v] = self.sigmoid((self.w.T).dot(self.X_train[v]))
            E_grad = (self.X_train.T).dot(y-self.Y_train)+self.reg_vec.dot(self.w)
            rnn = np.identity(len(self.X_train))
            np.fill_diagonal(rnn, ys * (1 - ys))
            E_hess = (self.X_train.T).dot(rnn).dot(self.X_train)+self.reg_vec
            w_new = self.w - np.linalg.inv(E_hess).dot(E_grad)
            E = self.error(self.Y_train, ys)
            print(E)
            if E_old-E < self.tol:
                print('w:', self.w)
                break
            E_old = E
            self.w = w_new          

    def predict(self, x_test):
        self.X_test = x_test
        if self.add_intercept:
            self.X_test = np.concatenate((self.X_test, np.ones((self.X_test.shape[0],1))), axis=1)
        self.ypred = np.zeros(len(self.X_test)) 
        for i in range(len(self.X_test)):
            prob = 1/(1+np.exp(-np.transpose(self.w).dot(self.X_test[i])))
            if prob >= 0.5:
                self.ypred[i] = 1
            else:
                self.ypred[i] = 0
        
        return self.ypred
    
    def accuracy(self, ypred, y):
        return (np.sum(ypred == y)/len(y))
        
        

In [17]:
# Case 1: lambda = 1 for all coefficients
dim=len(X_train[0])
lambda_vec = np.identity(dim+1)
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(X_train, Y_train)
ypred = logic1.predict(X_test)
print('accuracy = ', logic1.accuracy(ypred, Y_test))

21190.890427137954
11576.24536514505
11760.705919498994
w: [0.04799818508019224 1.5389440482709194e-06 0.34758857329256726
 0.00041941350396767693 0.0014799999357623438 0.05451097197749426
 0.5939155867404171 -0.9455666273351457 -0.4935382758896178
 0.08255523295991862 -1.5728875324764438 -1.204258817808504
 -2.209707944314312 -1.0674280403633867 -0.9537588984112493
 -0.7978832755714089 -0.2220909770289466 -0.5176595791152978
 -1.4702542591150765 -1.23502785257377 -0.9608838906853316
 -0.6365170928434858 0.06720851692502922 1.8843603820826145
 -0.9863510528816284 0.7580870899427488 -0.7736176398524585
 1.757589219782966 -0.5952610267258418 -1.6851089483497979
 1.5908324216956098 0.7544012477959043 -1.2766805804369352
 -2.1664743498710326 -1.6371653661466774 -1.3292928012823721
 -0.444345315559948 -0.4749852121793774 1.3451262448699643
 -2.2954826824779233 -1.5729024204352497 -1.1045686535292984
 -1.262296772149066 -0.9740077243716128 0.604566266005609
 0.7303965661824356 0.165144455224

In [18]:
# Case 2: lambda = 1 for all but the intercept, no regularization for intcercept term.
dim=len(X_train[0])
lambda_vec = np.identity(dim+1)
lambda_vec[dim][dim]=0

logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(X_train, Y_train)
ypred = logic1.predict(X_test)
print('accuracy = ', logic1.accuracy(ypred, Y_test))

21190.873237900432
11571.92427646343
11724.649904638907
w: [0.04884769809970042 1.5776826797947523e-06 0.5058246139265468
 0.00041928907600014 0.0014805531593458528 0.05513720705959371
 1.4768998295146885 -0.0625234998932647 0.3963602854404518
 0.9582980228694427 -0.6920955289787107 -0.32286587430780056
 -1.754073233939616 -0.28581879922712605 -0.3226149749171675
 -0.3289609299109768 1.077737098354347 0.6827021765721086
 -0.38755417122191654 -0.3114850277793185 -1.108463422022724
 -0.6290245170383465 -0.23995320563355638 1.1223818377609978
 -0.6649065657029481 0.2941451570213489 0.38663712125784255
 1.1449222618024981 -0.42974403453786303 -0.8452288135068267
 2.271810902964249 1.6396528393341383 -0.4501331098060564
 -1.312993073130984 -0.8015956202607379 -0.5015131212128234
 -0.015326226788240055 -0.04459998434407618 1.7700539698972197
 -1.8668497971362399 -1.1402591073528088 -0.6711109817525908
 -0.8249265445929045 -0.5693464697520758 1.0284189690229724
 1.157547044106936 0.5915640316

In [19]:
# Case 3: lambda = 1 for numerical-valued features, lambda = 0.5 for binary-valued features, no regularization for incercept term
dim = len(X_train[0])
lambda_vec=np.identity(dim+1)*0.5
for i in range(6):
    lambda_vec[i][i]=1
lambda_vec[dim][dim]=0
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(X_train, Y_train)
ypred = logic1.predict(X_test)
print('accuracy = ', logic1.accuracy(ypred, Y_test))

21190.552730655778
11569.450456413033
11710.43322612571
w: [0.04883671425526556 1.5805036930220293e-06 0.5006579487981361
 0.0004191453718509901 0.0014808698130147463 0.05510793895046128
 1.5676156695433776 0.02262930398355137 0.48322353800295853
 1.049956272497232 -0.6048811477419137 -0.23902574123653317
 -2.279517899570828 -0.31427235172776424 -0.3442993243837169
 -0.3498028468530813 1.0788678210159752 0.6622290366629772
 -0.42615011802227126 -0.3445836813207758 -1.1133204772186875
 -0.633756555893493 -0.23310658565565812 1.1692490505339728
 -0.6773256087961925 0.3107838747974534 0.4745436240718507
 1.1786208188703908 -0.43767668628412065 -0.8859054350551119
 2.530316638030813 1.5910307924895313 -0.49391799210148585
 -1.3530909643909494 -0.845318744730489 -0.5431143032233773
 0.038395408499985184 0.00788894794497072 1.8257988744752203
 -1.825144085731685 -1.092090743607607 -0.6196205469215295
 -0.7737883474422163 -0.5395943824911864 1.0803335481463836
 1.218386141552888 0.64607041066

In [ ]:
'''
Q2.4
'''

In [20]:
# split the training data into subtraining (90%) and tuning (10%) to search for the best hyper-parameters
from sklearn.model_selection import train_test_split
X_subtrain, X_tuning, Y_subtrain, Y_tuning = train_test_split(X_train, Y_train, test_size=0.1, random_state=78)
print(len(X_subtrain))
print(len(Y_subtrain))
print(len(X_tuning))
print(len(Y_tuning))

27145
27145
3017
3017


In [ ]:
'''
Q2.5 (20%) Use sklearn.linear_model.LogisticRegression to train and test the model (including hyper-parameter tuning). 
Compare the estimated parameters and test accuracy with those from your own models.
'''

In [21]:
from sklearn.linear_model import LogisticRegression

grids = [0.01*(10**(4/9))**i for i in range(10)]
temp = []
Y_train=Y_train.astype('int')
for s in grids:
    clf = (LogisticRegression(C=1/s, tol=1e-5, max_iter=1000, n_jobs=2, solver='newton-cg'))
    clf.fit(X_train, Y_train)
    ypred = clf.predict(X_test)
    accuracy = np.sum(ypred==Y_test)/len(Y_test)
    print("accuracy[%f]:"%s, accuracy)
    temp.append(accuracy)
s = grids[np.argmax(temp, axis=0)]
print("best parameter:", s)

accuracy[0.010000]: 0.796480743691899
accuracy[0.027826]: 0.7964143426294821
accuracy[0.077426]: 0.7964143426294821
accuracy[0.215443]: 0.7965471447543161
accuracy[0.599484]: 0.796613545816733
accuracy[1.668101]: 0.7966799468791501
accuracy[4.641589]: 0.798074369189907
accuracy[12.915497]: 0.799136786188579
accuracy[35.938137]: 0.802324037184595
accuracy[100.000000]: 0.8096945551128818
best parameter: 99.99999999999997
